In [8]:
# This script is used for interacting with OpenLCA.
# Codes are adapted from Julian Rickert @GreenDelta by Ao Chen @ETH.
# Connect to IPC server in OpenLCA software before running. (Port: 8080)
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {'highlight_selected_word': {
    'highlight_across_all_cells': True,
    'only_cells_in_scroll': False,
    'delay': 500,
    'code_cells_only': True,
}})


import pandas as pd
import olca
import uuid
import math
from datetime import datetime

from matplotlib import pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import rcParams
import matplotlib.patches as mpatches
import seaborn as sns

params = {'mathtext.default': 'regular' }

client = olca.Client(8080)
client

In [2]:
# Get all process dataset in database
process_descriptor = client.get_descriptors(olca.Process)
process_list = []
id_list = []
location_list = []

for process in process_descriptor:
    process_list.append(process.name)
    id_list.append(process.id)
    location_list.append(process.location)
processes_df = pd.DataFrame(list(zip(process_list,
                                   id_list, location_list)),
                               columns=['name', 'id', 'location'])
# processes_df

In [20]:
process_to_search_df = pd.read_excel('/Users/ocean/Desktop/RA-SustainableConstruction/OpenLCA/List_of_Materials_ecoinvent.xlsx', header=None)


'concrete block production | concrete block | Cutoff, U'

In [37]:
# Search specific process databases with key word(s)
# Note that the search is blankspace-sensitive
def process_search(process_to_search_df):
    for i in range(len(process_to_search_df)):
        search_process_df = processes_df[processes_df['name'].str.contains(process_to_search_df.iloc[i, 0], regex=False)]# TODO: change keywords
search_process = []
large_df = pd.concat(
    list(some_function_that_yields_dataframes()), ignore_index=True)


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,[concrete block production | concrete block | ...,"[clay brick production | clay brick | Cutoff, ...",None,"[natural stone plate production, cut | natural...","[flat glass production, coated | flat glass, c...","[cement production, Portland | cement, Portlan...","[sand quarry operation, open pit mine | sand |...","[gravel production, crushed | gravel, crushed ...",[reinforcing steel production | reinforcing st...,"[glued laminated timber production, MUF-glue |...",...,"[plywood production | plywood | Cutoff, U, afa...",[glass wool mat production | glass wool mat | ...,[polystyrene foam slab for perimeter insulatio...,"[polystyrene production, extruded, CO2 blown |...","[polyurethane production, rigid foam | polyure...",None,"[roof tile production | roof tile | Cutoff, U,...",[gypsum plasterboard production | gypsum plast...,"[natural stone plate production, cut | natural...",[ceramic tile production | ceramic tile | Cuto...
1,[concrete block production | concrete block | ...,"[clay brick production | clay brick | Cutoff, ...",None,"[natural stone plate production, cut | natural...","[flat glass production, coated | flat glass, c...","[cement production, Portland | cement, Portlan...",None,"[gravel production, crushed | gravel, crushed ...",[reinforcing steel production | reinforcing st...,None,...,"[plywood production | plywood | Cutoff, U, 302...",[glass wool mat production | glass wool mat | ...,[polystyrene foam slab for perimeter insulatio...,"[polystyrene production, extruded, CO2 blown |...","[polyurethane production, rigid foam | polyure...",None,"[roof tile production | roof tile | Cutoff, U,...",[gypsum plasterboard production | gypsum plast...,"[natural stone plate production, cut | natural...",[ceramic tile production | ceramic tile | Cuto...
2,[concrete block production | concrete block | ...,None,None,None,None,"[cement production, Portland | cement, Portlan...",None,"[gravel production, crushed | gravel, crushed ...",[reinforcing steel production | reinforcing st...,None,...,"[plywood production | plywood | Cutoff, U, 9b9...",[glass wool mat production | glass wool mat | ...,[market for polystyrene foam slab for perimete...,"[polystyrene production, extruded, CO2 blown |...",None,None,None,None,None,None
3,None,None,None,None,None,"[cement production, Portland | cement, Portlan...",None,"[gravel production, crushed | gravel, crushed ...",None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,"[cement production, Portland | cement, Portlan...",None,"[gravel production, crushed | gravel, crushed ...",None,None,...,None,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,"[cement production, Portland | cement, Portlan...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,None,None,None,None,None,"[cement production, Portland | cement, Portlan...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,None,None,None,None,None,"[cement production, Portland | cement, Portlan...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,"[cement production, Portland | cement, Portlan...",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [4]:
# Get all flow dataset in database
flow_descriptor = client.get_descriptors(olca.Flow)
flow_list = []
id_list = []

for flow in flow_descriptor:
    flow_list.append(flow.name)
    id_list.append(flow.id)
flows_df = pd.DataFrame(list(zip(flow_list,
                                   id_list)),
                               columns=['name', 'id'])
# flows_df

In [5]:
# Search specific flow databases with key word(s)
# Note that the search is blankspace-sensitive
search_flow_df = flows_df[flows_df['name'].str.contains(r'oil(?!$)')] # TODO: change keywords
search_flow_df.reset_index(drop=True, inplace=True)

# search_flow_df

In [9]:
# Create product systems for selected processes
search_list_processID = list(search_process_df['id'])
product_system_ids = []
product_system_names = []
product_system_locations = pd.Series(search_process_df['location'])

# for i in range(len(search_list_processID)):
for i in range(2): # for test
    print('Creating product system of: '+search_process_df['name'][i]+' | '+product_system_locations[i])
    product_system = client.create_product_system(search_list_processID[i], default_providers='prefer', preferred_type='UNIT_PROCESS') #TODO: preferred_type can be changed to "SYSTEM_PROCESS" if needed
    product_system_ids.append(product_system.id)
    ps = client.find(olca.ProductSystem, search_process_df['name'][i])
    product_system_names.append(ps.name)
    # print('Remaining number of product systems to be created: '+str(len(search_list_processID)-i-1))
    print('Remaining number of product systems to be created: '+str(2-i-1)) # for test

product_system_df = pd.DataFrame(list(zip(product_system_names, product_system_ids)), 
                    columns=['product_system_name', 'product_system_id'])

product_system_df.assign(product_system_location=product_system_locations)

Creating product system of: electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CA-SK
Remaining number of product systems to be created: 1
Creating product system of: electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | NL
Remaining number of product systems to be created: 0


,product_system_name,product_system_id,product_system_location
0,"electricity, high voltage, production mix | el...",70b1ff14-8fec-442c-811c-1338ca6723f6,CA-SK
1,"electricity, high voltage, production mix | el...",b7eab461-2c88-49bb-ab91-ffab019eca34,NL


In [82]:
setup = olca.CalculationSetup()
setup.calculation_type = olca.CalculationType.CONTRIBUTION_ANALYSIS # TODO: Can choose SIMPLE_CALCULATION, CONTRIBUTION_ANALYSIS, UPSTREAM_ANALYSIS, REGIONALIZED_CALCULATION, MONTE_CARLO_SIMULATION
setup.impact_method = client.find(olca.ImpactMethod, 'IPCC 2013') # TODO: choose impact method accordingly

# amount is the amount of the functional unit (fu) of the system that
# should be used in the calculation; unit, flow property, etc. of the fu
# can be also defined; by default openLCA will take the settings of the
# reference flow of the product system
setup.amount = 1.0

In [109]:
# for ps in range(len(product_system_names)):
for ps in range(2): # for test
    
    print('Working on '+product_system_names[ps]+' | '+product_system_locations[ps]+' | '+product_system_ids[ps])

    setup.product_system = client.get(olca.ProductSystem, uid=product_system_ids[ps])

    print(setup.product_system.id)

    calc_result = client.calculate(setup)

    client.excel_export(calc_result, 'calc_result/'+product_system_locations[ps]+'_'+'calc_result.xlsx')

    print('Remaining number of processes to be calculated: '+str(len(product_system_names)-ps-1))

Working on electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | CA-SK | 3b8f9ea4-3c76-44a0-9c5c-2e31cf4bb429
3b8f9ea4-3c76-44a0-9c5c-2e31cf4bb429
Remaining number of processes to be calculated: 1
Working on electricity, high voltage, production mix | electricity, high voltage | Cutoff, U | NL | 1a21b9f1-ece7-4537-bb2f-3a391f556da0
1a21b9f1-ece7-4537-bb2f-3a391f556da0
Remaining number of processes to be calculated: 0


In [177]:
GWP100a_df = pd.DataFrame()
# for i in range(len(product_system_locations)):
for i in range(2): # for test
    filename_xlsx = 'calc_result/'+product_system_locations[i]+'_calc_result.xlsx'
    # filename_xlsx = 'calc_result/'+product_system_locations[0]+'_calc_result.xlsx' # for test
    results_db = pd.ExcelFile(filename_xlsx)
    impact_df = results_db.parse('Impacts')
    setup_df = results_db.parse('Calculation setup')

    new_header = impact_df.iloc[0] #grab the first row for the header
    impact_df = impact_df[1:] #take the data less the header row
    impact_df.columns = new_header #set the header row as the df header

    GWP100a_series = pd.Series(impact_df.iloc[2, 2:])
    GWP100a_series['Process'] = search_process_df['name'][i]
    GWP100a_series['location_extended'] = setup_df.iloc[3, 2]

    GWP100a_df[product_system_locations[i]] = GWP100a_series

GWP100a_df.T.to_excel('GWP100a_summary.xlsx')


/Users/ocean/opt/miniconda3/envs/suscons/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
